---
title: "Online Feature Serving"
date: 2021-04-25
type: technical_note
draft: false
---

## Connect to the Feature Store

In [1]:
import hsfs

connection = hsfs.connection()
fs = connection.get_feature_store()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
31,application_1624276539905_0011,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

## Get training dataset

In [2]:
td_meta = fs.get_training_dataset("card_fraud_model_holuser20", 1)

In [3]:
td_meta.show(10)

+---------------+---------------+----------------+----------------+--------------+-----------------+-----------------+-----------------+-----------------+
|avg_amt_per_10m|avg_amt_per_12h|stdev_amt_per_1h|num_trans_per_1h|avg_amt_per_1h|num_trans_per_12h|stdev_amt_per_12h|num_trans_per_10m|stdev_amt_per_10m|
+---------------+---------------+----------------+----------------+--------------+-----------------+-----------------+-----------------+-----------------+
|        2187.16|       863.5691|        548.8967|               3|        360.56|               22|        1866.4856|                1|              NaN|
|           97.6|      1092.6254|             NaN|               1|        279.99|               26|        2200.1265|                1|              NaN|
|        4859.49|      1273.4976|             NaN|               1|        796.41|               16|        2138.3506|                1|              NaN|
|          89.22|      163.36412|             NaN|               1|   

To identify which primary key names are used for this training dataset query use serving_keys method

In [4]:
#`init_prepared_statement` method is needed to get serving_keys in case `get_serving_vector` has not beed called yet. This is not necessary for `get_serving_vector` method itself
td_meta.init_prepared_statement() 
td_meta.serving_keys

{'cc_num'}

In [5]:
td_meta.show(1)

+---------------+---------------+----------------+----------------+--------------+-----------------+-----------------+-----------------+-----------------+
|avg_amt_per_10m|avg_amt_per_12h|stdev_amt_per_1h|num_trans_per_1h|avg_amt_per_1h|num_trans_per_12h|stdev_amt_per_12h|num_trans_per_10m|stdev_amt_per_10m|
+---------------+---------------+----------------+----------------+--------------+-----------------+-----------------+-----------------+-----------------+
|        2187.16|       863.5691|        548.8967|               3|        360.56|               22|        1866.4856|                1|              NaN|
+---------------+---------------+----------------+----------------+--------------+-----------------+-----------------+-----------------+-----------------+
only showing top 1 row

For demo purposes lets prepare list of primary key values that we are interested in to buils feature vectore from online feature store

In [6]:
card_numbers = ["4867010117638802",
                 "4564139086560436", 
                 "4638396144844325",
                 "4460285888258185", 
                 "4032763187099525", 
                 "4650661577010550",
                 "4925013053127624"
                ]

In [9]:
card_numbers = ['4369075906669451',
 '4023558713593186',
 '4897249935555210',
 '4056057388547395',
 '4382106450108944']

In [10]:
for card_number in card_numbers:
    serving_vector = td_meta.get_serving_vector({'cc_num': card_number})
    print (serving_vector)

[]
[]
[]
[]
[]

In [11]:
dir(td_meta)

['ENTITY_TYPE', 'EXTERNAL', 'HOPSFS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_coalesce', '_data_format', '_description', '_feature_store_id', '_features', '_from_query', '_id', '_infer_training_dataset_type', '_label', '_location', '_name', '_prepared_statement_connection', '_prepared_statements', '_querydto', '_seed', '_serving_keys', '_splits', '_statistics_config', '_statistics_engine', '_storage_connector', '_training_dataset_api', '_training_dataset_engine', '_training_dataset_type', '_version', 'add_tag', 'coalesce', 'compute_statistics', 'data_format', 'delete', 'delete_tag', 'description', 'feature_store_id', 'from_response_json', 'get_query', 'get_serving_vector', 'get_statist

In [18]:
o=td_meta.tf_data

In [25]:
td_meta.tf_data(target_name = "avg_amt_per_10m").tf_record_dataset(batch_size=1, num_epochs=1, process=True)

An error was encountered:
hdfs://10.0.0.216:8020/Projects/kafka/kafka_Training_Datasets/card_fraud_model_holuser20_1/card_fraud_model_holuser20/part-00000-0b85d394-6302-40a6-bb93-4b2f7d4124a9-c000.tfrecord; Unknown error 255
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hsfs/core/tfdata_engine.py", line 152, in tf_record_dataset
    self._input_files, self._cycle_length
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hsfs/core/tfdata_engine.py", line 376, in _get_tf_dataset
    dataset, input_files
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hsfs/core/tfdata_engine.py", line 330, in _create_tfrecord_feature_description
    example = self._return_example_tf2(dataset)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hsfs/core/tfdata_engine.py", line 323, in _return_example_tf2
    for raw_record in dataset.take(1):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-p